In [1]:
import pandas as pd
import numpy as np
import pickle
import re

In [2]:
# 2. Carreguem dades
df = pd.read_csv("data/datosFinal.csv")
df

,Startup,Company,Category,Investment,Investors,Date,URL,Name,Description,Address,...,Technologies,Other fields,Funding stage,Founded,Employees,Business model,Target,Spinoff participants,capital_prev,investors
0,EQUITO,EQUITO APP S.L,FinancingProptech,0.3M €,Lanzadera,June 2024,startup/barcelona/equito/5843,EQUITO APP S.L,A real estate investment platform enabling use...,Paseig De Gracia 19 08007 Barcelona,...,"Fintech & Insurtech, \r\n\t\t \...",Social Economy,Seed (0.25 M EUR - < 1 M EUR),2021,From 11 to 20,Marketplace,Consumer,None / Not a spinoff,0.3M € (JUNE 2024),Lanzadera
1,VORA,VORA,Fashion & Design,0.3M €,Casilda Güell i Javier Sobrino,June 2024,startup/barcelona/vora/6706,VORA,"Vora is inspired by the city of Barcelona, pro...","Calle Cavallers, 54 P. 4 Pta. 1 08034 Barcelona",...,E-commerce,Social Economy,Seed (0.25 M EUR - < 1 M EUR),2022,Undisclosed,-,Consumer,None / Not a spinoff,0.3M € (JUNE 2024),Casilda Güell i Javier Sobrino
2,LEXDOKA,SEEDFY ECOSYSTEM S.L.,Business Services & SoftwareLegaltech,0.3M €,Codols Finances,May 2024,startup/barcelona/lexdoka/6563,SEEDFY ECOSYSTEM S.L.,LexDoka is a collaborative platform that turns...,"Calle Pallars, 108 08018 Barcelona",...,"Fintech & Insurtech, \r\n\t\t \...",Industry 4.0,Seed (0.25 M EUR - < 1 M EUR),2021,Undisclosed,Saas,Business,None / Not a spinoff,0.3M € (MAY 2024),Codols Finances
3,KOKUAI,KOKUAI S.L.,HealthtechBusiness Services & Software,0.3M €,"Elisabeth Martinez, Oriol Vila, Blast",September 2024,startup/barcelona/kokuai/5838,KOKUAI S.L.,Kokuai is a Barcelona based start-up founded i...,"Carrer Llacuna, 162 08018 Barcelona",...,"AI & Big Data, \r\n\t\t \t\t\r\...",-,Seed (0.25 M EUR - < 1 M EUR),2020,From 1 to 5,Saas,Business,None / Not a spinoff,0.3M € (SEPTEMBER 2024),"Elisabeth Martinez, Oriol Vila, Blast"
4,FLAMAID,FLAMAID S.L.,Hardware,0.3M €,ONA Capital,July 2024,startup/barcelona/flamaid/7137,FLAMAID S.L.,Flamaid is a safety device equipped with an au...,"Calle Salvador Espriu, 47 P. 5 Pta. 1 08005 Ba...",...,IoT & Sensors,Social Economy,Seed (0.25 M EUR - < 1 M EUR),2023,Undisclosed,Development & Manufacturing,Consumer,None / Not a spinoff,0.1M € (NOVEMBER 2024),NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,LAAGAM,CID BRAND SL,Fashion & Design,NaN,Tomas Meyer,October 2024,startup/barcelona/laagam/5599,CID BRAND SL,We inspire girls to take the world by storm.th...,"Pasaje Marimon, 23 Loc 08021 Barcelona",...,E-commerce,-,Seed (0.25 M EUR - < 1 M EUR),2016,From 6 to 10,Saas,-,None / Not a spinoff,(APRIL 2023),Cabiedes & Partners
88,LAAGAM,CID BRAND SL,Fashion & Design,NaN,Tomas Meyer,October 2024,startup/barcelona/laagam/5599,CID BRAND SL,We inspire girls to take the world by storm.th...,"Pasaje Marimon, 23 Loc 08021 Barcelona",...,E-commerce,-,Seed (0.25 M EUR - < 1 M EUR),2016,From 6 to 10,Saas,-,None / Not a spinoff,0.3M € (DECEMBER 2020),"cabiedes & Partners, Startupexplore"
89,LAAGAM,CID BRAND SL,Fashion & Design,NaN,Tomas Meyer,October 2024,startup/barcelona/laagam/5599,CID BRAND SL,We inspire girls to take the world by storm.th...,"Pasaje Marimon, 23 Loc 08021 Barcelona",...,E-commerce,-,Seed (0.25 M EUR - < 1 M EUR),2016,From 6 to 10,Saas,-,None / Not a spinoff,0.4M € (JULY 2018),n.a.
90,RIMBARLAT,RIMBARLAT S.L.,HealthtechSports,NaN,Barça Innovation Hub,October 2024,startup/barcelona/rimbarlat/5928,RIMBARLAT S.L.,Regenerative medicine for sports muscle injuries,"Avinguda De La Riera De Cassoles, 43 4 08012 B...",...,"Personalized medicine, \r\n\t\t ...",-,Seed (0.25 M EUR - < 1 M EUR),2020,Undisclosed,Others,Business,Technological Centre (BIOCAT),(OCTOBER 2024),Barça Innovation Hub


### Preprocessing

1. Company

In [35]:
df['es_sociedad_limitada'] = np.where(df["Company"].str.strip().str.endswith('S.L'), 'Sí', 'No')

2. Category

3. Investment

In [36]:
df["Investment"] = df["Investment"].astype(str).str.replace("€", "", regex=False)
df["Investment"] = df["Investment"].astype(str).str.replace("M", "", regex=False)
df["Investment"] = df["Investment"].astype("float")*1000000

4. Investor

In [37]:
df["Investors"].value_counts()

Investors
Tomas Meyer                                    16
Fundadors                                       8
ONA Capital                                     8
Barça Innovation Hub                            8
Codols Finances                                 4
Casilda Güell i Javier Sobrino                  4
Lanzadera                                       4
Capital Cell                                    4
Fundació Telefónica                             4
Elisabeth Martinez, Oriol Vila, Blast           4
Grow Venture Partners                           4
Estats Units                                    4
tres efes' (familiars, amics i entusiastes)     4
BcombinatorFI Group                             4
Business Angels                                 4
Name: count, dtype: int64

5. Date

In [38]:
df["Mes Inversion"] = [mes.split(" ")[0] for mes in df["Date"]]
df["Year Inversion"] = [mes.split(" ")[1] for mes in df["Date"]]

## Eliminar variable Date

6. Address

Sacamos las coordenadas a partir de la dirección

In [97]:
def limpiar_direccion(dir_raw):
    dir_limpia = dir_raw
    
    # Elimina paréntesis y su contenido
    dir_limpia = re.sub(r"\(.*?\)", "", dir_limpia)
    
    # Elimina referencias tipo P., Pta., Planta, Puerta, Loc, Sector, Sobreático, Num, Tienda
    dir_limpia = re.sub(r"\b(P\. |Pta\.?|Planta|Puerta|Loc|Sector|Sobreático|Num|Tienda)\b\.?\s*\d*\w*", "", dir_limpia, flags=re.IGNORECASE)
    
    # Elimina referencias tipo P., Pta., Planta, Puerta, Loc, Sector, Sobreático, Num, Tienda
    dir_limpia = re.sub(r"C/", "Carrer", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Urbanizacion", "Urbanizacio", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Rbla", "Rambla", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Plaza", "Plaça", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Pasaje", "Passatge", dir_limpia, flags=re.IGNORECASE)
    dir_limpia = re.sub(r"Paseig", "Passeig", dir_limpia, flags=re.IGNORECASE)
    
    # Elimina códigos postales (5 dígitos)
    dir_limpia = re.sub(r"\b\d{5}\b", "", dir_limpia)
    
    # Elimina comas innecesarias y múltiples espacios
    dir_limpia = re.sub(r",", "", dir_limpia)
    dir_limpia = re.sub(r"\s+", " ", dir_limpia)
    
    # Limpieza final de espacios
    dir_limpia = dir_limpia.strip()
    
    return dir_limpia

In [63]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time

# Inicializa el geolocalizador
geolocator = Nominatim(user_agent="GetLoc")

# Función segura para geocodificar
def safe_geocode(direccion):
    try:
        location = geolocator.geocode(direccion, timeout=10)
        if location:
            latituds = location.latitude
            longituds = location.longitude
        else:
            print("No se encontró la dirección.")
            latituds = np.nan
            longituds = np.nan
    except GeocoderTimedOut:
        print("Timeout — intentando de nuevo...")
        time.sleep(1)
        return safe_geocode(direccion)
    except Exception as e:
        print("Error:", e)
    return (latituds, longituds)

In [86]:
direcciones = [re.sub("Calle", "Carrer", direc) for direc in df["Address"]]

In [98]:
direcciones_limpias = [limpiar_direccion(d) for d in direcciones]
direcciones_limpias

['Passeig De Gracia 19 Barcelona',
 'Carrer Cavallers 54 Barcelona',
 'Carrer Pallars 108 Barcelona',
 'Carrer Llacuna 162 Barcelona',
 'Carrer Salvador Espriu 47 Barcelona',
 'Carrer Salvador Espriu 47 Barcelona',
 'Carrer Pallars 73 Barcelona',
 'Plaça Pau Vila 1 C Barcelona',
 'Rambla Catalunya B Barcelona',
 'Carrer Via Augusta 35 Barcelona',
 'Via Augusta 195 Barcelona Catalonia Barcelona',
 'Carrer Calabria 35 Barcelona',
 'Carrer Salvador Espriu 47 Barcelona',
 'Carrer Salvador Espriu 47 Barcelona',
 'Avinguda Lleida 32 0 Bellpuig',
 'Urbanizacio Cabrils Ii 7- 8 Cabrils',
 'Carrer De La Rosella 17 Barcelona',
 'Passatge Marimon 23 Barcelona',
 'Passatge Marimon 23 Barcelona',
 'Passatge Marimon 23 Barcelona',
 'Passatge Marimon 23 Barcelona',
 'Avinguda De La Riera De Cassoles 43 4 Barcelona',
 'Avinguda De La Riera De Cassoles 43 4 Barcelona',
 'Passeig De Gracia 19 Barcelona',
 'Carrer Cavallers 54 Barcelona',
 'Carrer Pallars 108 Barcelona',
 'Carrer Llacuna 162 Barcelona',
 

In [101]:
# Llama a la función con tu dirección
latituds = [safe_geocode(dir)[0] for dir in direcciones]

No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró la dirección.
No se encontró

In [102]:
direcciones

['Paseig De Gracia 19 08007 Barcelona',
 'Carrer Cavallers, 54 P. 4 Pta. 1 08034 Barcelona',
 'Carrer Pallars, 108 08018 Barcelona',
 'Carrer Llacuna, 162 08018 Barcelona',
 'Carrer Salvador Espriu, 47 P. 5 Pta. 1 08005 Barcelona',
 'Carrer Salvador Espriu, 47 P. 5 Pta. 1 08005 Barcelona',
 'Carrer Pallars, 73 08018 Barcelona',
 'Plaza Pau Vila (Ed. Palau De Mar), 1 P. 2, Sector 2 C 08039 Barcelona',
 'Rbla Catalunya Num 78 Planta 3 Puerta 1B 08008 B 08008 Barcelona',
 'Carrer Via Augusta, 35 P. 6 08006 Barcelona',
 'Via Augusta 195 Barcelona Catalonia 08021 Barcelona',
 'C/ Calabria 35 Sobreático  2 08015 Barcelona',
 'Carrer Salvador Espriu, 47 P. 5 Pta. 1 08005 Barcelona',
 'Carrer Salvador Espriu, 47 P. 5 Pta. 1 08005 Barcelona',
 'Avinguda Lleida, 32  25250,0 Bellpuig',
 'Urbanizacion Cabrils Ii (crta Cabrils A Vilassar), Tienda, 7- 8 08348 Cabrils',
 'Carrer De La Rosella 17 08035 Barcelona',
 'Pasaje Marimon, 23 Loc 08021 Barcelona',
 'Pasaje Marimon, 23 Loc 08021 Barcelona',
 '

In [104]:
latituds

[nan,
 nan,
 41.3959298,
 41.4061236,
 nan,
 nan,
 41.3946751,
 nan,
 nan,
 nan,
 41.3977869,
 nan,
 nan,
 nan,
 nan,
 nan,
 41.4187409,
 nan,
 nan,
 nan,
 nan,
 41.4013789,
 41.4013789,
 nan,
 nan,
 41.3959298,
 41.4061236,
 nan,
 nan,
 41.3946751,
 nan,
 nan,
 nan,
 41.3977869,
 nan,
 nan,
 nan,
 nan,
 nan,
 41.4187409,
 nan,
 nan,
 nan,
 nan,
 41.4013789,
 41.4013789,
 nan,
 nan,
 41.3959298,
 41.4061236,
 nan,
 nan,
 41.3946751,
 nan,
 nan,
 nan,
 41.3977869,
 nan,
 nan,
 nan,
 nan,
 nan,
 41.4187409,
 nan,
 nan,
 nan,
 nan,
 41.4013789,
 41.4013789,
 nan,
 nan,
 41.3959298,
 41.4061236,
 nan,
 nan,
 41.3946751,
 nan,
 nan,
 nan,
 41.3977869,
 nan,
 nan,
 nan,
 nan,
 nan,
 41.4187409,
 nan,
 nan,
 nan,
 nan,
 41.4013789,
 41.4013789]